# Cooking Recipes dataset

The cooking recipes dataset is a collection of html webpage. On the webpage that host the dataset, we found two perl scripts: one that extract the indredient from the recipes and another that extract the nutrients from a recipe.
In order to use those scripts, we clean the log file that came with the data set to transform it into a list of http link. We then exectute those script with this cleaned file as input and redirect the output into another file that we format in order to use it.

## Cleaning msg.txt

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
log = open("data/msg.txt", mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
clean_log = open("data/msg_clean.txt", mode='w', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)

In [3]:
content = log.readlines()

In [4]:
for i in range(len(content)):
    line = content[i]
    breaked_line = line.split(' ')
    if len(breaked_line) == 1:
        breaked_line = line.split('\t')
        url = breaked_line[1]
        cleaned_url = url[4:]
        clean_log.write(cleaned_url)
        clean_log.write('\n')
    else:
        url = breaked_line[2]
        cleaned_url = url[4:]
        clean_log.write(cleaned_url)
        clean_log.write('\n')

## Cleaning nutrients.txt

The nutrients.txt file is the result of the extractNutrientsFromRecipes perl script when executed on the msg.txt file. It contains the nutrients of a recipe (in mg). Recipes are identified by a md5 encryptyon of their html. Therefore, their ids are the same than the one we can find in ingredients.txt

In [5]:
# Open the file
nutrients_txt = open("data/recipeClean/nutrients.txt", mode='r', buffering=-1, encoding="ISO-8859-1", errors=None, newline=None, closefd=True, opener=None)

In [6]:
# Get all the lines
content = nutrients_txt.readlines()

In [7]:
# Retrieve each nutrient and the id of the recipes
nutrients = []
ids = np.zeros(len(content), dtype=object)

def check_nan(nutrient):
    if(nutrient == '?' or nutrient == '-'):
        return np.nan
    else:
        return float(nutrient)

for i in range(len(content)):
    line = content[i].split('\t')
    nutrients.append((check_nan(line[0]),
                     check_nan(line[1]),
                     check_nan(line[2]),
                     check_nan(line[3]),
                     check_nan(line[4]),
                     check_nan(line[5])))
    ids[i] = line[6]

In [8]:
# Construct a DataFrame where nutrients are indexed by the ids of the recipes
nutrients = np.array(nutrients)
nutrients_df = pd.DataFrame(nutrients, columns= ('kcal', 'carb', 'fat', 'prot', 'sodium', 'chol'), index=ids)
#nutrients_df.to_json("nutrients_cookies_recipes.json")

### Light Analysis of the nutrients dataframe

In [9]:
nutrients_df.describe()

,kcal,carb,fat,prot,sodium,chol
count,34675.000000,34634.000000,34623.000000,34630.000000,33824.000000,33631.000000
mean,347.626311,140.007216,154.379675,56.039255,614.618764,63.253001
std,438.068277,233.083793,246.405880,73.188498,1937.704896,105.702694
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,158.000000,48.500000,45.450000,12.000000,147.000000,4.000000
50%,270.000000,102.400000,102.600000,28.800000,343.000000,35.000000
75%,422.050000,172.000000,193.500000,84.800000,714.000000,84.600000
max,14082.000000,11342.800000,11222.100000,2770.680000,113226.600000,4149.400000


In [10]:
# If values of all the different nutrients are set to 0, it should be NaN values instead
nutrients_df[(nutrients_df['kcal'] == 0) & 
             (nutrients_df['carb'] == 0) & 
             (nutrients_df['fat'] == 0) & 
             (nutrients_df['prot'] == 0) &
             (nutrients_df['sodium'] == 0) &
             (nutrients_df['chol'] == 0)] = np.nan

In [11]:
nutrients_df.describe()

,kcal,carb,fat,prot,sodium,chol
count,34637.000000,34596.000000,34585.000000,34592.000000,33786.000000,33593.000000
mean,348.007689,140.160999,154.549299,56.100815,615.310041,63.324552
std,438.157086,233.165552,246.488042,73.205102,1938.684615,105.741041
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,158.300000,48.800000,45.900000,12.000000,148.000000,4.100000
50%,270.000000,102.400000,102.600000,29.200000,344.000000,35.000000
75%,422.500000,172.000000,193.500000,84.800000,715.000000,85.000000
max,14082.000000,11342.800000,11222.100000,2770.680000,113226.600000,4149.400000


In [12]:
nutrients_df.isnull().sum()

kcal      29612
carb      29653
fat       29664
prot      29657
sodium    30463
chol      30656
dtype: int64

If we take the minumum NaN values among the different kind of nutrients and we divide it by the total number of row we get: 29612/64249 = 0.46. We are close to the 47% of nil values announced [here](http://infolab.stanford.edu/~west1/from-cookies-to-cooks/)